# Example - Convert dataset to raster (GeoTiff)

Often, it is desirable to take a variable (band) out of your dataset and export it to a raster.
This is possible with the [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster) method. It does most of the work for you so you don't
have to.

Note: The [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster) method only works on a 2-dimensional or 3-dimensional `xarray.DataArray`

In [1]:
import rioxarray
import rasterio
import xarray
import re

See docs for [rioxarray.open_rasterio](../rioxarray.rst#rioxarray-open-rasterio)

In [2]:
rds = rioxarray.open_rasterio(
    "../../test/test_data/input/PLANET_SCOPE_3D.nc",
    variable=["green"],
)
rds

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29 2016-12-29 12:52:41.659696
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
    spatial_ref  int64 0
Data variables:
    green        (time, y, x) float64 ...
Attributes:
    coordinates:  spatial_ref

In [35]:
%timeit rds.green.rio.to_raster("green_band.tif", tiled=True, compress="LZW", windowed=True)

14.5 ms ± 462 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
!ls -lh green_band.tif

-rw-rw-r-- 1 snowal snowal 4.7K Jan 22 15:25 green_band.tif


###### rds.green.shape, rds.green.rio.crs, rds.green.rio.nodata, rds.green.rio.bounds()

In [37]:
%timeit rds = rasterio.open("green_band.tif"); rds.read()

3.53 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
default: 1.1M 10ms; 2.32ms;
zstd level 5: 2.3K 12 ms; 2.67 ms;
lzma: 2.6K 39 ms; 7ms;

## Managing Information Loss with xarray operations

Sometimes, you can lose important information from your dataset when performing operations.
You will likely want to keep track of your `nodata` and your `CRS`.

API Reference:

- [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster)
- [rio.write_crs()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.write_crs)
- [rio.update_attrs()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.update_attrs)
- [rio.crs](../rioxarray.rst#rioxarray.rioxarray.RasterArray.crs)
- [rio.nodata](../rioxarray.rst#rioxarray.rioxarray.RasterArray.nodata)

In [ ]:
new_ds = rds.green + rds.green
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

In [ ]:
new_ds.rio.to_raster("combination.tif")

In [ ]:
!rio info combination.tif

In [ ]:
new_ds = rds.green + rds.green
new_ds.rio.write_crs(rds.green.rio.crs, inplace=True)
new_ds.rio.update_attrs(rds.green.attrs, inplace=True)
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

In [ ]:
new_ds.rio.to_raster("combination_keep_attrs.tif")

In [ ]:
!rio info combination_keep_attrs.tif